In [1]:
import anndata as ad 

In [2]:
state_Data = ad.read_h5ad("/work/home/cryoem666/xyf/temp/pycharm/state/gene_perturnb_state/data/replogle.h5ad")
squidiff_Data = ad.read_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/diff_data/diff_train.h5ad")
import toml 
config = toml.load("/work/home/cryoem666/xyf/temp/pycharm/state/gene_perturnb_state/config_toml/hepg2.toml")

In [4]:
state_Data.obs['cell_line'].astype('category').cat.codes.values

array([0, 0, 0, ..., 3, 3, 3], dtype=int8)

In [3]:
import pickle 
file = pickle.load(open("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours3/vocab_state.pkl","rb"))

In [4]:
import sample_squidiff_ours
sampler = sample_squidiff_ours.sampler(
        model_path = '/work/home/cryoem666/xyf/temp/pycharm/Squidiff/Squidiff/checkpoints/our_hepg2_step400k/model.pt',
        perturb_len = len(file['perturb']),
        batch_len = len(file['batch']),
        cell_line_len = len(file['cell_line']),
        #gene_size = sum(state_Data.var['highly_variable'])
        gene_size=2000,
        diffusion_steps = 50000
    )

load model and diffusion...
{'output_dim': 2000, 'num_layers': 3, 'num_channels': 128, 'dropout': 0.0, 'class_cond': False, 'use_checkpoint': False, 'use_scale_shift_norm': True, 'use_fp16': False, 'use_encoder': True, 'state_dataset_config': {'perturb_len': 2014, 'batch_len': 56, 'cell_line_len': 4, 'gene_size': 2000, 'output_dim': 2000}, 'learn_sigma': False, 'diffusion_steps': 50000, 'noise_schedule': 'linear', 'timestep_respacing': '', 'use_kl': False, 'predict_xstart': False, 'rescale_timesteps': False, 'rescale_learned_sigmas': False, 'data_path': '', 'schedule_sampler': 'uniform', 'lr': 0.0001, 'weight_decay': 0.0, 'lr_anneal_steps': 100000.0, 'batch_size': 16, 'microbatch': -1, 'ema_rate': '0.9999', 'log_interval': 10000.0, 'save_interval': 10000.0, 'resume_checkpoint': '', 'fp16_scale_growth': 0.001, 'use_ddim': True, 'logger_path': '', 'model_path': '/work/home/cryoem666/xyf/temp/pycharm/Squidiff/Squidiff/checkpoints/our_hepg2_step400k/model.pt', 'comb_num': 1, 'drug_dimensio

In [7]:
import numpy as np 
def one_hot(label, key):
    vec = np.zeros(len(file[key]), dtype=int)
    idx = file[key][label]
    vec[idx] = 1
    return vec

In [5]:
test_gene = config['fewshot']['replogle_proper.hepg2']['val']+config['fewshot']['replogle_proper.hepg2']['test']

In [6]:
state_Data = state_Data[state_Data.obs['gene'].isin(file['perturb'])]

In [7]:
len(file['perturb'])

2014

In [8]:
state_Data.obs['perturb_label'] = state_Data.obs['gene'].apply(lambda x: file['perturb'][x])
state_Data.obs['batch_label'] = state_Data.obs['gem_group'].apply(lambda x: file['batch'][x])
state_Data.obs['cell_line_label'] = state_Data.obs['cell_line'].apply(lambda x: file['cell_line'][x])

/tmp/ipykernel_182516/1927203975.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  state_Data.obs['perturb_label'] = state_Data.obs['gene'].apply(lambda x: file['perturb'][x])


In [10]:
cell_type_list = ['jurkat', 'k562', 'rpe1']

In [11]:
case = state_Data[(state_Data.obs['cell_line']=='k562')&(state_Data.obs['gene']=='ATP6V1G1')]

In [12]:
case.shape

(65, 6642)

In [13]:
import torch
import numpy as np
z_sem = sampler.model.encoder.forward(
    torch.tensor(case[:, case.var['highly_variable']].X.toarray()).to('cuda'),
    perturb_label = torch.tensor(np.stack(case.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
    batch = torch.tensor(np.stack(case.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
    cell_type = torch.tensor(np.stack(case.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
)

scrnas_pred = sampler.pred(z_sem, gene_size = 2000)
     

In [22]:
z_sem

tensor([[ 0.0203,  0.0158, -0.0354,  ...,  0.0120,  0.0551,  0.0964],
        [ 0.0575, -0.0182,  0.0713,  ...,  0.0399,  0.0608,  0.0372],
        [ 0.0231,  0.0071,  0.0395,  ...,  0.0057, -0.0552,  0.0540],
        ...,
        [ 0.0485,  0.0080, -0.0607,  ...,  0.0418,  0.0051,  0.0186],
        [ 0.0005, -0.0489, -0.0211,  ...,  0.0093, -0.0056,  0.0500],
        [ 0.0254, -0.0005,  0.0585,  ...,  0.0069, -0.0471,  0.0281]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
r2_, pearsonr_ = sampler.cal_metric(scrnas_pred, case[:, case.var['highly_variable']] )

In [21]:
case[:, case.var['highly_variable']].X.toarray().mean(axis=0)

array([0.10916708, 0.16049147, 0.14435525, ..., 4.806311  , 3.00477   ,
       1.8745427 ], dtype=float32)

In [16]:
scrnas_pred.mean(axis=0)

tensor([33.9990, 52.1245, 14.1376,  ..., 15.0438, 45.7694, 58.7176],
       device='cuda:0')

In [15]:
r2_, pearsonr_

(-2.5045480728149414, 0.46383628)

In [ ]:
sample.

tensor([[0.0000e+00, 0.0000e+00, 3.5339e+00,  ..., 5.5611e+01, 8.9266e+01,
         1.1413e+00],
        [1.5325e+02, 1.4744e+02, 6.0450e+01,  ..., 0.0000e+00, 6.3629e+00,
         6.2667e+01],
        [0.0000e+00, 0.0000e+00, 3.7328e+01,  ..., 9.5615e+01, 1.0391e+02,
         8.9030e+01],
        ...,
        [9.1964e+01, 1.0995e+01, 4.5070e+01,  ..., 1.1042e+01, 9.0227e+00,
         0.0000e+00],
        [1.1343e+02, 1.7101e+02, 0.0000e+00,  ..., 4.5847e+01, 3.1564e+01,
         1.3830e-01],
        [2.1413e+00, 1.9505e+02, 4.5516e+01,  ..., 2.3828e+01, 4.8855e-01,
         1.6155e+02]], device='cuda:0')

In [ ]:
#拿ttf2作为例子
import torch
perturb, control_label = "ATP6V1G1", "non-targeting"
perturb_state_data = state_Data[state_Data.obs['gene'] == perturb]
perturb_direction_dict = {}
perturb_profile_dict = {}
control_profile_dict = {}
z_perturb_dict = {}
z_control_dict = {}
from tqdm import tqdm 
for cell_type in tqdm(cell_type_list, position=0):
    print(f"Processing cell type: {cell_type}")
    celltype_perturb_Data = perturb_state_data[perturb_state_data.obs['cell_line'] == cell_type]
    control_Data = state_Data[(state_Data.obs['gene'] == control_label)&(state_Data.obs['cell_line'] == cell_type)]
    print(f"Cell type: {cell_type}, Perturbation data shape: {celltype_perturb_Data.shape}, Control data shape: {control_Data.shape}")
    if celltype_perturb_Data.shape[0] == 0 or control_Data.shape[0] == 0:
        print(f"Skipping cell type: {cell_type} due to insufficient data.")
        continue
    z_perturb = sampler.model.encoder.forward(
        x_start = torch.tensor(celltype_perturb_Data[:, state_Data.var['highly_variable']].X.toarray(), dtype=torch.float32).to('cuda'),
        perturb_label = torch.tensor(np.stack(celltype_perturb_Data.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        batch = torch.tensor(np.stack(celltype_perturb_Data.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
        cell_type = torch.tensor(np.stack(celltype_perturb_Data.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
    )
    print(f"Cell type: {cell_type}, Perturbation latent shape: {z_perturb.shape}")
    z_control = sampler.model.encoder.forward(
        x_start = torch.tensor(control_Data[:, state_Data.var['highly_variable']].X.toarray(), dtype=torch.float32).to('cuda'),
        perturb_label = torch.tensor(np.stack(control_Data.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        batch = torch.tensor(np.stack(control_Data.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
        cell_type = torch.tensor(np.stack(control_Data.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
    )
    print(f"Cell type: {cell_type}, Perturbation latent shape: {z_perturb.shape}, Control latent shape: {z_control.shape}")
    direction = z_perturb.mean(dim=0) - z_control.mean(dim=0)
    print(f"Cell type: {cell_type}, Direction shape: {direction.shape}")
    perturb_direction_dict[cell_type] = direction
    z_perturb_dict[cell_type] = z_perturb
    z_control_dict[cell_type] = z_control
    perturb_profile_dict[cell_type] = celltype_perturb_Data
    control_profile_dict[cell_type] = control_Data
    



  0%|          | 0/3 [00:00<?, ?it/s]

Processing cell type: jurkat
Cell type: jurkat, Perturbation data shape: (234, 6642), Control data shape: (12013, 6642)
Cell type: jurkat, Perturbation latent shape: torch.Size([234, 60])


 33%|███▎      | 1/3 [00:00<00:00,  3.76it/s]

Cell type: jurkat, Perturbation latent shape: torch.Size([234, 60]), Control latent shape: torch.Size([12013, 60])
Cell type: jurkat, Direction shape: torch.Size([60])
Processing cell type: k562
Cell type: k562, Perturbation data shape: (32, 6642), Control data shape: (10691, 6642)
Cell type: k562, Perturbation latent shape: torch.Size([32, 60])


 67%|██████▋   | 2/3 [00:00<00:00,  4.07it/s]

Cell type: k562, Perturbation latent shape: torch.Size([32, 60]), Control latent shape: torch.Size([10691, 60])
Cell type: k562, Direction shape: torch.Size([60])
Processing cell type: rpe1
Cell type: rpe1, Perturbation data shape: (174, 6642), Control data shape: (11485, 6642)
Cell type: rpe1, Perturbation latent shape: torch.Size([174, 60])


100%|██████████| 3/3 [00:00<00:00,  4.07it/s]

Cell type: rpe1, Perturbation latent shape: torch.Size([174, 60]), Control latent shape: torch.Size([11485, 60])
Cell type: rpe1, Direction shape: torch.Size([60])


In [24]:
z_perturb_dict['k562'].shape, z_control_dict['k562'].shape

(torch.Size([32, 60]), torch.Size([10691, 60]))

In [35]:
# 看看latent在k562上的重建效果
sample_interp_k562_perturb = sampler.pred(
    z_sem = z_control_dict['k562'],
    gene_size = 2000
)
r2_, pearsonr_ = sampler.cal_metric(sample_interp_k562_perturb,control_profile_dict['k562'][:, state_Data.var['highly_variable']] )

In [36]:
r2_, pearsonr_ 

(-7.032767295837402, 0.4544764)

In [32]:
perturb_profile_dict['k562'][:, state_Data.var['highly_variable']]

View of AnnData object with n_obs × n_vars = 32 × 2000
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_hvg'

In [29]:
sample_interp_k562_perturb

tensor([[101.4783, 120.1826,  20.5033,  ...,  82.9439,  55.3134,   1.6676],
        [ 29.6133,  60.3053,  39.1713,  ...,   0.0000,  28.6653, 234.5341],
        [147.6770,   0.0000,  48.8272,  ...,  43.3956, 125.2541, 116.1456],
        ...,
        [ 80.0923,   2.0000,  87.8860,  ...,  49.3407,   0.4834,  13.7775],
        [  0.0000, 113.3022,  60.3742,  ...,   0.6529,   0.0000,   0.0000],
        [192.6514, 227.2016,  41.1534,  ...,   0.2463,  21.1935,   0.0000]],
       device='cuda:0')

In [53]:
file_test_gene = tuple(config['fewshot'].items())[0][1]['test']
hepg2_gene = set(state_Data[state_Data.obs['cell_line']=='hepg2'].obs['gene'])
test_gene = list(set(file_test_gene) & hepg2_gene)

In [51]:
test_gene

['MRPL39',
 'GPN3',
 'COG4',
 'YARS2',
 'BRIX1',
 'NHLRC2',
 'SDHAF2',
 'NUDCD3',
 'EMC1',
 'PPP6C',
 'SPOUT1',
 'STRIP1',
 'COPS3',
 'SNRNP48',
 'ZDHHC7',
 'MDN1',
 'PIAS1',
 'DHPS',
 'DENR',
 'LAS1L',
 'ISCA2',
 'STIL',
 'TPRKB',
 'RPS28',
 'RPP14',
 'RRP15',
 'USP10',
 'POLR1E',
 'UBA2',
 'IST1',
 'NVL',
 'RPS8',
 'RPP21',
 'MRPS23',
 'RPS20',
 'ZC3H13',
 'RPAP2',
 'XPO5',
 'TSEN2',
 'INTS10',
 'RCL1',
 'CENPN',
 'CCP110',
 'OPA1',
 'SOD2',
 'HSD17B10',
 'ISG20L2',
 'PES1',
 'CS',
 'MSL1',
 'TSFM',
 'NUTF2',
 'RPL26',
 'PTMA',
 'MED7',
 'TINF2',
 'ITGB1BP1',
 'CPSF6',
 'MED29',
 'NOM1',
 'NLE1',
 'EIF2S1',
 'MRPS16',
 'ANKRD49',
 'BAG6',
 'COMTD1',
 'ZBTB17',
 'UROD',
 'TAF10',
 'MIS18BP1',
 'VPS13D',
 'KDM6A',
 'FAU',
 'NAA15',
 'NSUN4',
 'MRPL9',
 'RAC1',
 'TXNL4B',
 'BAP1',
 'NDUFA6',
 'ARGLU1',
 'SEPSECS',
 'DNM1L',
 'CENPC',
 'SRP14',
 'DDX20',
 'MED26',
 'FGFR1OP',
 'GAR1',
 'UTP15',
 'NIFK',
 'POLL',
 'ZNF720',
 'TMEM214',
 'OSTC',
 'EFR3A',
 'SNRPC',
 'NBAS',
 'SNAPC5',
 'TR

In [ ]:
actual_test_gene = config

In [54]:
#cosine similarity between different cell types
from sklearn.metrics.pairwise import cosine_similarity
cell_types = list(perturb_direction_dict.keys())
directions = torch.stack([perturb_direction_dict[ct] for ct in cell_types]).detach().cpu().numpy()
similarity_matrix = cosine_similarity(directions)

In [56]:
perturb_direction_dict

{'k562': tensor([-0.0015,  0.0029,  0.0071,  0.0007,  0.0020, -0.0023, -0.0042, -0.0123,
          0.0032,  0.0018, -0.0095,  0.0009, -0.0063, -0.0009, -0.0026,  0.0048,
         -0.0019, -0.0015,  0.0024,  0.0016, -0.0011, -0.0015, -0.0033, -0.0059,
          0.0041, -0.0009, -0.0018, -0.0022, -0.0023, -0.0033, -0.0028, -0.0003,
         -0.0025,  0.0031, -0.0008,  0.0001, -0.0026, -0.0038,  0.0020, -0.0012,
          0.0022, -0.0013, -0.0090, -0.0049,  0.0026,  0.0012, -0.0039,  0.0016,
         -0.0006, -0.0035, -0.0103, -0.0024,  0.0070, -0.0021,  0.0007,  0.0003,
          0.0003, -0.0019,  0.0041,  0.0049], device='cuda:0',
        grad_fn=<SubBackward0>),
 'jurkat': tensor([-1.3440e-03,  9.6145e-04,  1.3870e-03,  1.0526e-03, -3.6950e-05,
         -2.8875e-03, -1.3636e-03, -1.9511e-03,  2.7316e-03, -1.5402e-03,
          4.1045e-05, -4.1175e-04, -3.2817e-04, -1.2531e-03, -1.4815e-03,
          2.5750e-03, -1.4945e-03, -5.4494e-04,  1.5465e-03, -2.3818e-03,
          3.0849e-04, -

In [55]:
similarity_matrix

array([[ 0.99999994,  0.4493432 , -0.54073685,  0.3392582 ],
       [ 0.4493432 ,  0.99999976, -0.3435316 ,  0.12021162],
       [-0.54073685, -0.3435316 ,  1.0000001 , -0.64503545],
       [ 0.3392582 ,  0.12021162, -0.64503545,  1.0000001 ]],
      dtype=float32)

In [ ]:
test_gene = 

In [ ]:
test_ct = tuple(config['fewshot'].keys())[0].split('.')[1]
control_label = 'non-targeting'
test_perturb = tuple(config['fewshot'].items())[0][1]['test']
for 

In [ ]:
sampler.model.encoder.forward()

In [5]:
import torch

# 加载 checkpoint（如果显存有限可以用 map_location="cpu"）
ckpt = torch.load("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours/model.pt",
                  map_location="cpu")

# 1. 如果是直接保存的 state_dict
if isinstance(ckpt, dict) and all(isinstance(v, torch.Tensor) for v in ckpt.values()):
    keys = ckpt.keys()
# 2. 如果 checkpoint 是个 dict，里头有 'state_dict' 或类似 key
elif isinstance(ckpt, dict) and "state_dict" in ckpt:
    keys = ckpt["state_dict"].keys()
else:
    # 其他情况，先看一下顶层 keys
    print("checkpoint 顶层 keys:", ckpt.keys())
    # 假设第一个 key 对应的就是参数 dict
    first = next(iter(ckpt.values()))
    if isinstance(first, dict):
        keys = first.keys()
    else:
        raise ValueError("无法识别 checkpoint 格式，请手动检查结构")

# 打印所有参数名
print("模型包含的权重层／参数名称：")
for k in keys:
    print(k)

模型包含的权重层／参数名称：
time_embed.0.weight
time_embed.0.bias
time_embed.2.weight
time_embed.2.bias
mlp_blocks.0.fc1.weight
mlp_blocks.0.fc1.bias
mlp_blocks.0.fc2.weight
mlp_blocks.0.fc2.bias
mlp_blocks.0.layer_norm1.weight
mlp_blocks.0.layer_norm1.bias
mlp_blocks.0.layer_norm2.weight
mlp_blocks.0.layer_norm2.bias
mlp_blocks.0.time_dense.weight
mlp_blocks.0.time_dense.bias
mlp_blocks.1.fc1.weight
mlp_blocks.1.fc1.bias
mlp_blocks.1.fc2.weight
mlp_blocks.1.fc2.bias
mlp_blocks.1.layer_norm1.weight
mlp_blocks.1.layer_norm1.bias
mlp_blocks.1.layer_norm2.weight
mlp_blocks.1.layer_norm2.bias
mlp_blocks.1.time_dense.weight
mlp_blocks.1.time_dense.bias
mlp_blocks.2.fc1.weight
mlp_blocks.2.fc1.bias
mlp_blocks.2.fc2.weight
mlp_blocks.2.fc2.bias
mlp_blocks.2.layer_norm1.weight
mlp_blocks.2.layer_norm1.bias
mlp_blocks.2.layer_norm2.weight
mlp_blocks.2.layer_norm2.bias
mlp_blocks.2.time_dense.weight
mlp_blocks.2.time_dense.bias
input_layer.weight
input_layer.bias
output_layer.weight
output_layer.bias
